<h1 style = "text-align: center">Collaborative Filtering Data Model</h1>

<h3 style = "text-align: center">Food.com Recipe Recommender - SOEN 471 (Big Data Analytics)</h3>

## Objective:
The objective of this notebook is to create a recommender system data model that recommend recipes based on user preferences using collaborative filtering.

In [145]:
import os
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn
from sklearn.metrics.pairwise import cosine_similarity
import warnings

warnings.filterwarnings("ignore") # ignore error of displot

# accessing directory
for dirname, _, filenames in os.walk('./clean_data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

./clean_data/interactions_TRAIN.csv
./clean_data/interactions_TEST.csv
./clean_data/recipes.csv


## Reading files:

In [146]:
# read
training = pd.read_csv("./clean_data/interactions_TRAIN.csv")
testing = pd.read_csv("./clean_data/interactions_TEST.csv")
recipes = pd.read_csv("./clean_data/recipes.csv")

# print shapes of data
print("Shape of training model: ", training.shape)
print("Shape of testing model: ", testing.shape)
print("Shape of recipes model: ", recipes.shape)

Shape of training model:  (1019129, 6)
Shape of testing model:  (113237, 6)
Shape of recipes model:  (231636, 19)


## Since the data is big, we will take a small random sample:

In [147]:
interactions_sample = training.sample(40000)
interactions_sample.head(1)

,Unnamed: 0,user_id,recipe_id,date,rating,review
836811,37715,226863,312378,2008-11-12,3,"I followed this recipe exactly, so I'm not sur..."


In [148]:
recipes_sample = recipes.sample(40000)
recipes_sample.head(1)

,Unnamed: 0,name,recipe_id,minutes,contributor_id,submitted,tags,n_steps,steps,description,ingredients,n_ingredients,Calories,Total_fat_PDV,Sugar_PDV,Sodium_PDV,Protein_PDV,Saturated_fat_PDV,Carbohydrates_PDV
171807,55160,red achiote mexican rice,67319,40,80138,2003-07-24,"['60-minutes-or-less', 'time-to-make', 'course...",13,"['rinse rice and drain well', 'set apart', 'pr...",a tomato-less mexican red rice. popular in the...,"['white rice', 'white onion', 'garlic', 'achio...",10,275.9,15.0,2.0,16.0,11.0,7.0,13.0


## Join both samples based on 

In [149]:
data = pd.merge(interactions_sample, recipes_sample, right_on='recipe_id', left_on='recipe_id')
print("The shape of the joind training data sample: ", data.shape)
data.head(2)

The shape of the joind training data sample:  (6902, 24)


,Unnamed: 0_x,user_id,recipe_id,date,rating,review,Unnamed: 0_y,name,minutes,contributor_id,...,description,ingredients,n_ingredients,Calories,Total_fat_PDV,Sugar_PDV,Sodium_PDV,Protein_PDV,Saturated_fat_PDV,Carbohydrates_PDV
0,119776,121690,206272,2007-03-19,5,I cooked the rice in my electric rice steamer ...,13638,denise s saffron vegetable fried rice,25,341142,...,an interesting meld of indian and chinese cook...,"['long grain rice', 'water', 'powdered saffron...",12,212.8,10.0,4.0,5.0,7.0,4.0,11.0
1,95472,858469,154636,2009-09-29,0,This was really good. If you are a banana brea...,50557,weight watcher 1 point banana bread flex points,70,7108,...,"moist and tasty! for those on ww, it's an eas...","['bananas', 'splenda sugar substitute', 'natur...",7,87.8,0.0,30.0,10.0,3.0,0.0,6.0


 ## Summary of the descriptive statistics include those that summarize the central tendency, dispersion and shape of a dataset's distribution:

In [150]:
data.describe()

,Unnamed: 0_x,user_id,recipe_id,rating,Unnamed: 0_y,minutes,contributor_id,n_steps,n_ingredients,Calories,Total_fat_PDV,Sugar_PDV,Sodium_PDV,Protein_PDV,Saturated_fat_PDV,Carbohydrates_PDV
count,6902.000000,6.902000e+03,6902.000000,6902.000000,6902.000000,6902.000000,6.902000e+03,6902.000000,6902.000000,6902.000000,6902.000000,6902.000000,6902.000000,6902.000000,6902.000000,6902.000000
mean,114025.756013,1.353601e+08,162404.523327,4.399305,28473.455665,126.593306,5.033479e+06,9.787163,9.158650,455.150377,33.693567,77.478557,31.110693,36.587946,42.027673,14.774703
std,66067.211579,4.962649e+08,131522.569956,1.278239,16805.580110,1772.470220,9.427315e+07,5.811689,3.728647,574.326555,50.566635,233.306537,65.488555,41.695716,65.829974,27.298191
min,174.000000,1.533000e+03,39.000000,0.000000,32.000000,0.000000,1.530000e+03,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,56957.000000,1.354002e+05,53914.000000,4.000000,14114.750000,25.000000,3.744900e+04,6.000000,6.000000,186.650000,9.000000,9.000000,6.000000,7.000000,8.000000,4.000000
50%,112315.000000,3.360580e+05,123864.500000,5.000000,28815.000000,40.000000,8.983100e+04,8.000000,9.000000,318.850000,20.000000,24.000000,16.000000,21.000000,24.000000,9.000000
75%,172183.250000,8.156715e+05,249890.750000,5.000000,42641.750000,70.000000,2.503322e+05,13.000000,11.000000,517.400000,39.000000,68.000000,36.000000,56.000000,51.000000,16.000000
max,227289.000000,2.002372e+09,536524.000000,5.000000,58760.000000,129615.000000,2.001488e+09,70.000000,26.000000,9038.500000,900.000000,5174.000000,2361.000000,872.000000,1241.000000,484.000000


## Creates a pivot table from the data and replace any missing value by 0:

In [151]:
pivot_table = data.pivot_table(index='recipe_id', columns='user_id', values='rating')
pivot_table.fillna(0, inplace=True)

## Define a function that generates similar recipes based on recipe_id provided

In [152]:
def similar_recipes(recipe_id):
    recipe = pivot_table.loc[recipe_id].values.reshape(1,-1)
    cosine_similarities = cosine_similarity(pivot_table, recipe).flatten()
    related_recipe_indices = cosine_similarities.argsort()[:-6:-1]
    related_recipes = recipes.loc[related_recipe_indices][['name', 'ingredients']]
    return related_recipes.values.tolist()

## Define a function that returns a recommended recipe based on minutes and calories provided 

In [153]:
def recommend_recipe(minutes, calories):
    # find the recipes that have similar minutes and calories as the input
    similar_minutes = data[(data['minutes'] >= minutes-10) & (data['minutes'] <= minutes+10)]
    similar_calories = similar_minutes[(similar_minutes['Calories'] >= calories-100) & (similar_minutes['Calories'] <= calories+100)]
    recipe_ids = similar_calories['recipe_id'].unique().tolist()

    # recommend similar recipes for each recipe in the filtered data
    recommended_recipes = []
    for recipe_id in recipe_ids:
        recommended_recipes.extend(similar_recipes(recipe_id))

    # remove duplicates
    recommended_recipes = list(set([tuple(recipe) for recipe in recommended_recipes]))

    return recommended_recipes[:10] # return top 10 recommended recipes

## Usage Example

In [ ]:
recommended_recipes = recommend_recipe(30, 500)
print("Recommended Recipes that needs 30 minutes to prepare and contains 500 calories:")

for i, recipe in enumerate(recommended_recipes):
    print(f"{i+1}. Recipe Name: {recipe[0]}")
    print("Ingredients:")
    ingredients = recipe[1].split(', ')
    for ingredient in ingredients:
        print("- ", ingredient)
    print()